1.


In [2]:
import numpy as np
import pandas as pd
import cv2
import pickle
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import normalize
from google.colab import files

# Upload CSV file
uploaded = files.upload()

# Import pickle module
import pickle

# Load data
data = pd.read_csv(next(iter(uploaded)))

# Image Feature Extraction


Saving A2_Data.csv to A2_Data.csv


In [7]:
# data = pd.read_csv("data.csv")

# Function to extract data from specified rows
def extract_data(row_numbers, dataframe):
    extracted_data = []
    for row_number in row_numbers:
        row_data = dataframe.iloc[row_number][["Image", "Review Text"]].tolist()
        extracted_data.append(row_data)
    return extracted_data

# Example usage
row_numbers = [0, 3, 5]  # Specify the row numbers you want to extract
extracted_data = extract_data(row_numbers, data)
print(extracted_data[0])

["['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']", 'Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.']


2

In [8]:
import numpy as np
import pandas as pd
import cv2
import pickle
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import normalize
from google.colab import files
# data = pd.read_csv("A2_Data.csv")
import os
import pandas as pd
import urllib.request
from PIL import Image
from io import BytesIO
import shutil

# Function to create folder for storing images
def create_image_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Delete folder and its contents if it already exists
    os.makedirs(folder_path)  # Create new folder

# Function to fetch and save an image
def fetch_and_save_image(url, image_path):
    try:
        with urllib.request.urlopen(url) as response:
            image_data = response.read()
            image = Image.open(BytesIO(image_data))
            image.save(image_path)
            print(f"Image saved successfully: {image_path}")
    except Exception as e:
        print(f"Error downloading image: {url}, {e}")

# Function to fetch and save images from a list of URLs
def fetch_and_save_images(url_list, folder_path, index):
    for i, url in enumerate(url_list):
        image_path = os.path.join(folder_path, f"image_{index}_{i}.jpg")
        fetch_and_save_image(url, image_path)

# Function to process data and fetch images
def process_data_and_fetch_images(data, folder_path):
    create_image_folder(folder_path)
    for index, row in data.iterrows():
        image_urls = eval(row["Image"])
        fetch_and_save_images(image_urls, folder_path, index)

# Example usage
data = pd.read_csv("A2_Data.csv")
folder_path = "extracted_images"
process_data_and_fetch_images(data, folder_path)


Image saved successfully: extracted_images/image_0_0.jpg
Image saved successfully: extracted_images/image_1_0.jpg
Image saved successfully: extracted_images/image_1_1.jpg
Image saved successfully: extracted_images/image_1_2.jpg
Image saved successfully: extracted_images/image_2_0.jpg
Image saved successfully: extracted_images/image_3_0.jpg
Image saved successfully: extracted_images/image_4_0.jpg
Image saved successfully: extracted_images/image_5_0.jpg
Image saved successfully: extracted_images/image_5_1.jpg
Image saved successfully: extracted_images/image_5_2.jpg
Image saved successfully: extracted_images/image_5_3.jpg
Image saved successfully: extracted_images/image_6_0.jpg
Image saved successfully: extracted_images/image_7_0.jpg
Image saved successfully: extracted_images/image_7_1.jpg
Image saved successfully: extracted_images/image_7_2.jpg
Image saved successfully: extracted_images/image_7_3.jpg
Image saved successfully: extracted_images/image_8_0.jpg
Image saved successfully: extra

In [9]:
import shutil

source_folder = "extracted_images"
destination_folder = "new_folder"

shutil.copytree(source_folder, destination_folder)

'new_folder'

3

In [10]:
import numpy as np
import os
import pickle
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

def extract_image_features(folder_path):
    # Load pre-trained ResNet50 model
    base_model = ResNet50(weights='imagenet')
    # Remove the classification head
    model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

    # Get list of files in the folder and sort them
    files = sorted(os.listdir(folder_path))

    # Extract features from images
    image_features = {}
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(folder_path, file)
            img = Image.open(image_path)
            img = img.resize((224, 224))  # Resize image to match ResNet50 input size
            img_array = np.array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)  # Preprocess image for ResNet50
            features = model.predict(img_array)
            image_features[file] = features.flatten()

    return image_features

def normalize_image_features(image_features):
    normalized_features = {}
    for file, features in image_features.items():
        normalized_features[file] = features / np.linalg.norm(features)
    return normalized_features

# Example usage
folder_path = "new_folder"
image_features = extract_image_features(folder_path)
normalized_features = normalize_image_features(image_features)

# Save normalized image features along with file names using Pickle
with open('image_features_resnet.pkl', 'wb') as f:
    pickle.dump(normalized_features, f)


1/1 [==============================] - 0s 327ms/step


4

In [11]:
import numpy as np
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import defaultdict
import math
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# Download NLTK resources


# Load data
data = pd.read_csv("A2_Data.csv")

# Preprocess text data
def preprocess_text(text):
    # Check if the text is a string
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()

        # Remove special characters (excluding alphanumeric characters and spaces)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]

        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        return tokens
    else:
        return []

# Calculate TF-IDF scores

def calculate_tf_idf(data):
    # Document frequency (DF) dictionary
    df = defaultdict(int)
    # Term frequency (TF) dictionary for each document
    tf = {}
    # Total number of documents
    total_docs = len(data)

    # Calculate DF and TF
    for i, row in data.iterrows():
        # print(i)
        if(i==828):
          tf[i] = {"":0}
          continue
        tokens = preprocess_text(row["Review Text"])
        if tokens:
            # Update document frequency
            for term in set(tokens):
                df[term] += 1

            # Calculate TF for each term in the document
            term_freq = {}
            max_freq = 0
            for term in tokens:
                term_freq[term] = term_freq.get(term, 0) + 1
                max_freq = max(max_freq, term_freq[term])
            tf[i] = {term: freq / (len(tokens)+1)  for term, freq in term_freq.items()}

        df[""]=0




    # Calculate IDF and TF-IDF
    idf = {}
    tf_idf = {}
    for term, freq in df.items():
        idf[term] = math.log(total_docs / (freq + 1))  # Add smoothing to avoid division by zero
    for doc_id, term_freq in tf.items():
        tf_idf[doc_id] = {term: freq * idf[term] for term, freq in term_freq.items()}



    return tf_idf
def normalize_tf_idf(tf_idf):
    normalized_tf_idf = {}
    for doc_id, scores in tf_idf.items():
        max_score = max(scores.values())
        min_score = min(scores.values())
        if max_score != min_score:
            normalized_scores = {term: (score - min_score) / (max_score - min_score) for term, score in scores.items()}
        else:
            normalized_scores = {term: 0 for term in scores.keys()}  # Set all scores to 0 if max_score equals min_score
        normalized_tf_idf[doc_id] = normalized_scores
    return normalized_tf_idf

# Calculate TF-IDF scores
tf_idf_scores = calculate_tf_idf(data)

# Normalize TF-IDF scores
# normalized_tf_idf_scores = normalize_tf_idf(tf_idf_scores)

# Print TF-IDF scores for the first document
print("TF-IDF scores for the first document:")
for term, score in tf_idf_scores[0].items():
    print(f"{term}: {score}")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


TF-IDF scores for the first document:
love: 0.1294670857519162
vintag: 0.49541453747696207
spring: 0.6035392171627877
strat: 0.19522285281649224
good: 0.09874244438703475
tension: 0.29440816885286986
great: 0.07101963474075758
stabil: 0.29440816885286986
float: 0.3450913073663904
bridg: 0.2095254510932952
want: 0.13517644691368044
way: 0.16273063661112297
go: 0.13046710708606876


In [12]:
tf_idf_scores[828]


{'': 0.0}

In [13]:
with open('tf_idf_scores.pkl', 'wb') as f:
    pickle.dump(tf_idf_scores, f)

with open('tf_idf_scores.pkl', 'rb') as f:
    tf_idf_scores=pickle.load(f)

In [14]:
import os
import shutil

# Create input_folder if it doesn't exist
input_folder_path = 'input_folder'
if not os.path.exists(input_folder_path):
    os.makedirs(input_folder_path)

# Define source and destination paths
source_image_path = 'extracted_images/image_4_0.jpg'
destination_image_path = os.path.join(input_folder_path, 'image_4_0.jpg')

# Copy image from extracted_images folder to input_folder
shutil.copyfile(source_image_path, destination_image_path)

print("Image copied successfully to input_folder.")


Image copied successfully to input_folder.


5

In [15]:
import os
import urllib.request

# Function to download and save an image from a URL
def download_image(url, filename):
    try:
        with urllib.request.urlopen(url) as response:
            image_data = response.read()
            with open(filename, 'wb') as f:
                f.write(image_data)
        print(f"Image saved successfully: {filename}")
    except Exception as e:
        print(f"Error downloading image: {url}, {e}")

# URL of the image to download
image_url = "https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg"

# Folder path to save the image
folder_path = "input_images"

# Ensure the folder exists, create it if not
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define the new filename
new_filename = "input_image1.jpg"

# Download and save the image
download_image(image_url, os.path.join(folder_path, new_filename))


Image saved successfully: input_images/input_image1.jpg


6

In [16]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted image features
with open('image_features_resnet.pkl', 'rb') as f:
    image_features = pickle.load(f)

# print(image_features.shape)

def extract_image_features(file):
    # Load pre-trained ResNet50 model
    base_model = ResNet50(weights='imagenet')
    # Remove the classification head
    model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

    # Extract features from images
    image_features = []

    if file.endswith(".jpg"):
        # image_path = os.path.join(folder_path, file)
        img = Image.open(file)
        img = img.resize((224, 224))  # Resize image to match ResNet50 input size
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)  # Preprocess image for ResNet50
        features = model.predict(img_array)
        image_features.append(features.flatten())

    return np.array(image_features)


def normalize_image_features(image_features):
    normalized_features = normalize(image_features, axis=1)
    return normalized_features
import numpy as np

# Function to compute cosine similarity between two vectors
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

# Function to find top 3 file names with highest cosine similarity

def top_3_similarity(input_vector, feature_dict):
    y = set()
    similarities = {}
    for filename, feature_vector in feature_dict.items():
        similarity = cosine_similarity(input_vector, feature_vector)
        parts = filename.split("_")

        if parts[1] in similarities:
          similarities[parts[1]] = max(similarity, similarities[parts[1]])
          continue
        similarities[parts[1]] = similarity



    top_3 = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:3]
    # y.add()

    return top_3, similarities

# Example usage
input_features = extract_image_features("input_images/input_image1.jpg")
input_features = normalize_image_features(input_features)
# # input_features_array = np.array(input_features).reshape(1, -1)  # Reshape to match dataframe format
input_vector = input_features[0]
# input_vector =

top_3, img_similar = top_3_similarity(input_vector, image_features)
print("Top 3 file names with highest cosine similarity(Image):")
row_numbers = [0, 3, 5]  # Specify the row numbers you want to extract
i=0
for filename, similarity in top_3:
    row_numbers[i] =  int(filename)
    print(filename, ":", similarity)
    i+=1


def extract_data(row_numbers, dataframe):
    extracted_data = []
    for row_number in row_numbers:
        row_data = dataframe.iloc[row_number][["Image", "Review Text"]].tolist()
        extracted_data.append(row_data)
    return extracted_data

# Example usage

extracted_data = extract_data(row_numbers, data)

print(extracted_data[0])



1/1 [==============================] - 1s 1s/step
Top 3 file names with highest cosine similarity(Image):
758 : 0.9859211
655 : 0.83618855
578 : 0.73849565
["['https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg']", 'I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.']


7

In [34]:
text = "Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go."
def preprocess_text(text):
    # Check if the text is a string
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()

        # Remove special characters (excluding alphanumeric characters and spaces)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]

        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        return tokens
    else:
        return []
# Function to compute cosine similarity between two vectors
import numpy as np

def cosine_similarity_text(vector1, vector2):
    # Determine the shorter and longer vectors
    shorter_vector = vector1 if len(vector1) <= len(vector2) else vector2
    longer_vector = vector2 if len(vector1) <= len(vector2) else vector1

    # Compute the dot product
    dot_product = 0
    for i in range(len(shorter_vector)):
        dot_product += shorter_vector[i] * longer_vector[i]

    # Compute the norms
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Handle division by zero
    if norm1 == 0 or norm2 == 0:
        return 0

    # Compute the cosine similarity
    similarity = dot_product / (norm1 * norm2)

    return similarity


input_review = preprocess_text(text)

def get_index(input_review):
  i1 =-1
  # print(input_review)
  d={}
  for term in input_review:
      d[term]=1

  input_review = list(d.keys())

  # print(input_review)

  for i in range(0, 1000):
      s1 = []  # Initialize s1 as an empty string
      # if()
      for term, score in tf_idf_scores[i].items():
          s1.append(term)

      # print()

      if s1 == input_review:
          i1 = i
          break

  input_vec = list(tf_idf_scores[i1].values())
  return input_vec



image_wali_text_similarity=[]

given_vec = get_index(input_review)
for items in extracted_data:
  t = preprocess_text(items[1])
  searc = get_index(t)
  image_wali_text_similarity.append(cosine_similarity_text(given_vec,searc))


print(extracted_data[0][0])
print(extracted_data[0][1])
print(top_3[0][1])
print(image_wali_text_similarity[0])
print((float(image_wali_text_similarity[0]) + float(top_3[0][1])) / 2)

print()

print(extracted_data[1][0])
print(extracted_data[1][1])
print(top_3[1][1])
print(image_wali_text_similarity[1])
print((float(image_wali_text_similarity[1]) + float(top_3[1][1])) / 2)

print()

print(extracted_data[2][0])
print(extracted_data[2][1])
print(top_3[2][1])
print(image_wali_text_similarity[2])
print((float(image_wali_text_similarity[2]) + float(top_3[2][1])) / 2)


['https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg']
I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
0.9859211
0.6504966010997913
0.81820884299031

['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']
These locking tuners look great and keep tune.  Good quality materials and construction.  Excellent upgrade to any guitar.  I had to drill additions holes for installation.  If your neck already comes with pre-drilled holes, then they should drop right in, otherwise you will need to buy a guitar tuner pin drill jig, also available from Amazon.
0.83618855
0.4544895176723459
0.64533903621807

['https://images-na.ssl-images-amazon.com/images/I/61n284XL9HL._SY88.jpg']
Easy as heck to put on, In my opinion better than sperzel. These took literally 10 minutes to put on my  MIM strat.
Only thing ill say is you will probably n

8

In [55]:

import pickle
with open('tf_idf_scores.pkl', 'rb') as f:
    tf_idf_scores=pickle.load(f)


# text = "Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go."
def preprocess_text(text):
    # Check if the text is a string
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()

        # Remove special characters (excluding alphanumeric characters and spaces)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]

        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        return tokens
    else:
        return []
# Function to compute cosine similarity between two vectors
import numpy as np

def cosine_similarity_text(vector1, vector2):
    # Determine the shorter and longer vectors
    shorter_vector = vector1 if len(vector1) <= len(vector2) else vector2
    longer_vector = vector2 if len(vector1) <= len(vector2) else vector1

    # Compute the dot product
    dot_product = 0
    for i in range(len(shorter_vector)):
        dot_product += shorter_vector[i] * longer_vector[i]

    # Compute the norms
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Handle division by zero
    if norm1 == 0 or norm2 == 0:
        return 0

    # Compute the cosine similarity
    similarity = dot_product / (norm1 * norm2)

    return similarity


# Function to find top 3 file names with highest cosine similarity
def top_3_similarity_text(input_vector, feature_dict):
    y = set()
    similarities = []
    similar = {}
    # i=0
    for i in range(len(tf_idf_scores)):
        feature_vector = list(tf_idf_scores[i].values())
        similarity = cosine_similarity_text(input_vector, feature_vector)
        similarities.append([similarity, i])
        similar[i] = similarity
        # if(similarity>1):
        #   print(similarity)


        # i+=1

    # print(similarities)

    top_3 = sorted(similarities)
    # print(top_3)
    # print()
    # y.add()

    return top_3, similar



def extract_data(row_numbers, dataframe):
    extracted_data = []
    for row_number in row_numbers:
        row_data = dataframe.iloc[row_number][["Image", "Review Text"]].tolist()
        extracted_data.append(row_data)
    return extracted_data

input_review = preprocess_text(text)
i1 =-1
# print(input_review)
d={}
for term in input_review:
    d[term]=1

input_review = list(d.keys())

print(input_review)

for i in range(0, 1000):
    s1 = []  # Initialize s1 as an empty string
    # if()
    for term, score in tf_idf_scores[i].items():
        s1.append(term)

    # print()

    if s1 == input_review:
        i1 = i
        break

# print(i1)

input_vec = list(tf_idf_scores[i1].values())


top3_text,txt_similar = top_3_similarity_text(input_vec, tf_idf_scores)
row_numbers = [0,3,5]
print("Top 3 file names with highest cosine similarity(text):")
for i in range(3):
  row_numbers[i] = top3_text[len(top3_text)-i-1][1]
  print(top3_text[len(top3_text)-i-1][1], top3_text[len(top3_text)-i-1][0])

# row_numbers[0] =5
extracted_data_list =extract_data(row_numbers, data)
first_images = extracted_data_list[0][0]
second_images = extracted_data_list[1][0]
third_images = extracted_data_list[2][0]
c_sim1 = 0
c_sim2 = 0
c_sim3 = 0

for i in range(int(len(first_images)/74)):
  c_sim1 += cosine_similarity(input_vector, image_features['image_'+str(row_numbers[0])+'_'+str(i)+'.jpg'])

c_sim1/= int(len(first_images)/74)

for i in range(int(len(second_images)/74)):
  c_sim2 += cosine_similarity(input_vector, image_features['image_'+str(row_numbers[1])+'_'+str(i)+'.jpg'])

c_sim2/= int(len(first_images)/74)

for i in range(int(len(third_images)/74)):
  c_sim3 += cosine_similarity(input_vector, image_features['image_'+str(row_numbers[2])+'_'+str(i)+'.jpg'])

c_sim3/= int(len(first_images)/74)

print(first_images)
print(extracted_data_list[0][1])
print(c_sim1)
print(top3_text[len(top3_text)-0-1][0])
print((float(c_sim1) + top3_text[len(top3_text)-0-1][0])/2)

print()

print(second_images)
print(extracted_data_list[1][1])
print(c_sim2)
print(top3_text[len(top3_text)-1-1][0])
print((float(c_sim2) + top3_text[len(top3_text)-1-1][0])/2)

print()

print(third_images)
print(extracted_data_list[2][1])
print(c_sim3)
print(top3_text[len(top3_text)-2-1][0])
print((float(c_sim3) + top3_text[len(top3_text)-2-1][0])/2)


['love', 'vintag', 'spring', 'strat', 'good', 'tension', 'great', 'stabil', 'float', 'bridg', 'want', 'way', 'go']
Top 3 file names with highest cosine similarity(text):
0 1.0000000000000002
353 0.9516145377685519
808 0.9254463649137965
['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
0.582210123538971
1.0000000000000002
0.7911050617694856

['https://images-na.ssl-images-amazon.com/images/I/81wpla0rPvL._SY88.jpg']
I use this with the Zoom G3n and it works well as an expression pedal. Good value, solid build quality, so far so good.
0.4395888149738312
0.9516145377685519
0.6956016763711915

['https://images-na.ssl-images-amazon.com/images/I/51lELCmVyNL._SY88.jpg']
This is a nice mic boom. I have tried a lightweight usb mic on this and I even put a MMX 990 mic on this an

In [ ]:
# img_similar["68"]
# txt_similar

new_sim =[]
for i in range(1000):
  if(str(i) not in img_similar.keys()):
    img_similar[str(i)]=0
  new_sim.append([(txt_similar[i]+img_similar[str(i)])/2.0,i])

new_sim = sorted(new_sim)

print("Top 3 file names with highest cosine similarity(composite):")

for i in range(3):
  print(new_sim[len(new_sim)-i-1][1], new_sim[len(new_sim)-i-1][0])

Top 3 file names with highest cosine similarity:
0 0.9925847351551057
205 0.7752596148680252
194 0.7494093034348366


In [1]:
!ls


sample_data
